In [2]:
import tensorflow as tf
import numpy as np
import time
import psutil
import os
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.models import Model
import pandas as pd

In [3]:
# Ensure TensorFlow only uses a limited amount of GPU memory
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Define batch sizes and number of runs
BATCH_SIZES = [1, 4, 8, 16, 32, 64]
NUM_RUNS = 10  # Runs per batch size

# Load NASNet model
base_model = NASNetMobile(weights='imagenet', input_shape=(224, 224, 3))

# Create a model that outputs intermediate layer results
layer_outputs = [layer.output for layer in base_model.layers]
model = Model(inputs=base_model.input, outputs=layer_outputs)

# Precompile TensorFlow functions for performance
@tf.function
def run_inference(input_tensor):
    return model(input_tensor)

2025-02-05 14:43:45.000175: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-02-05 14:43:45.000191: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-02-05 14:43:45.000195: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-02-05 14:43:45.000208: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-05 14:43:45.000216: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# Function to measure inference time and memory
def benchmark_model(device, batch_size):
    print(f"\nRunning on {device} with batch size {batch_size}")
    results = []

    for run in range(NUM_RUNS):
        # Create random input tensor
        input_tensor = tf.random.uniform((batch_size, 224, 224, 3), dtype=tf.float32)
        
        with tf.device(device):
            #layer_times = {}
            tensor_sizes = {}

            # Warm-up to avoid cold start latency
            _ = run_inference(input_tensor)

            start_time = time.perf_counter()

            # Measure Layer wise tensor size
            output = run_inference(input_tensor)
            
            # Measure per-layer execution time
            for idx, layer in enumerate(model.layers):
                # Calculate tensor size in MB (float32 = 4 bytes)
                tensor_size = np.prod(output[idx].shape) * 4 / (1024 ** 2)
                tensor_sizes[layer.name] = tensor_size
            
            total_time = time.perf_counter() - start_time
    
            results.append({
                "device": device,
                "batch_size": batch_size,
                "total_time": total_time
            })

    return results

## Experiment Logging

In [5]:
# Run benchmarks for CPU and GPU (if available)
devices = ["/CPU:0"]
if gpus:
    devices.append("/GPU:0")

final_results = []
for device in devices:
    for batch_size in BATCH_SIZES:
        try:
            tf.profiler.experimental.start(logdir = f"logs/Noise/Cache/{device[1:4]}/{batch_size}", 
                                           options=tf.profiler.experimental.ProfilerOptions(host_tracer_level = 3,
                                                                                            device_tracer_level = 3))
            final_results.extend(benchmark_model(device, batch_size))
        finally:
            tf.profiler.experimental.stop()


Running on /CPU:0 with batch size 1


2025-02-05 14:43:47.289082: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 14:43:47.289091: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2025-02-05 14:43:49.014830: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-02-05 14:43:50.411782: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 14:43:50.999991: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 14:43:51.000472: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/Cache/CPU/1/plugins/profile/2025_02_05_14_43_51/C17586.xplane.pb
2025-02-05 14:43:51.068855: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 14:43:51.068865: I external


Running on /CPU:0 with batch size 4


2025-02-05 14:43:54.707511: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 14:43:55.299793: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 14:43:55.300048: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/Cache/CPU/4/plugins/profile/2025_02_05_14_43_55/C17586.xplane.pb
2025-02-05 14:43:55.367429: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 14:43:55.367441: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.



Running on /CPU:0 with batch size 8


...

2025-02-05 14:46:05.734062: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 14:46:06.045752: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 14:46:06.046786: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/Cache/GPU/16/plugins/profile/2025_02_05_14_46_06/C17586.xplane.pb
2025-02-05 14:46:06.094983: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 14:46:06.095002: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.



Running on /GPU:0 with batch size 32


2025-02-05 14:46:16.509356: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 14:46:16.855877: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 14:46:16.856247: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/Cache/GPU/32/plugins/profile/2025_02_05_14_46_16/C17586.xplane.pb
2025-02-05 14:46:16.900561: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 14:46:16.900569: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.



Running on /GPU:0 with batch size 64


2025-02-05 14:47:02.655713: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 14:47:03.829423: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 14:47:03.830370: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/Cache/GPU/64/plugins/profile/2025_02_05_14_47_03/C17586.xplane.pb


## Results

In [6]:
df = pd.DataFrame(final_results)
df.head()

,device,batch_size,total_time
0,/CPU:0,1,0.042829
1,/CPU:0,1,0.042413
2,/CPU:0,1,0.042348
3,/CPU:0,1,0.040541
4,/CPU:0,1,0.039662


In [7]:
df.groupby(['device', 'batch_size']).agg({'total_time': ['min', 'mean', 'var', 'max']})

total_time                              
                         min      mean       var       max
device batch_size                                         
/CPU:0 1            0.039386  0.040884  0.000002  0.042829
       4            0.064078  0.067390  0.000012  0.074663
       8            0.086916  0.102506  0.000834  0.183935
       16           0.142225  0.149689  0.000059  0.160843
       32           0.283204  0.294445  0.000110  0.320506
       64           1.246195  1.454401  0.017527  1.650604
/GPU:0 1            0.089442  0.094301  0.000049  0.113257
       4            0.092283  0.096087  0.000038  0.113161
       8            0.093800  0.097852  0.000041  0.115525
       16           0.091539  0.107042  0.001950  0.232546
       32           0.141684  0.192252  0.010127  0.396843
       64           0.778337  1.193435  0.031663  1.377343